<a href="https://colab.research.google.com/github/cliche-niche/model-zoo-submissions/blob/main/ResNet/resNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers
import numpy as np

In [27]:
(trainIm, trainLab), (testIm, testLab) = cifar10.load_data()
trainIm = trainIm / 255.0
testIm = testIm / 255.0

trainImMean = np.mean(trainIm, axis=0)
trainIm -= trainImMean
testIm -= trainImMean

trainLab = tf.keras.utils.to_categorical(trainLab, 10)
testLab = tf.keras.utils.to_categorical(testLab, 10)

In [28]:
#For callbacks
class mcb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
       if(logs.get('accuracy')>0.75):
            print("\nReached 75% accuracy so cancelling training!")
            self.model.stop_training = True
cb = mcb()

https://www.tensorflow.org/guide/keras/custom_layers_and_models

Two classes, `resId` and `resSh`, with reference from the link above were written.

`resId` is used to make a resNet block in which Identity shortcuts are used, padding is used to keep dimensions of residual the same as input (inp). They are then added together and a LeakyReLU layer is applied on them

`resSh` is used to make a resNet block in which Shortcuts are used, the size of input is made the same as residual by using a 1x1 convolution with `strides= (2, 2)`. The residual is obtained by first using a `3x3 convolution` (with padding) and `strides= (2, 2)`, followed by another `3x3 convolution` (with padding) but `strides=1` this time.

A layer for `BatchNormalization`, followed by a `ReLU` is used after every convolution, unless they're being added, in which case `ReLU` is applied after their addition

In [30]:
class resId(layers.Layer):
    def __init__(self, filters=64, kerSize=3):
        super(resId, self).__init__()
        self.co = layers.Conv2D(filters, kernel_size=kerSize, padding='same',
                                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4))
        self.bn = layers.BatchNormalization()
        self.re = layers.Activation(activations.relu)
    def call(self, inp):
        x = inp
        id = inp
        x = self.co(x)
        x = self.bn(x)
        x = self.re(x)
        x = self.co(x)
        x = self.bn(x)
        return self.re(x+id)
 
class resSh(layers.Layer):
    def __init__(self, filters, kerSize=3, stride=(2, 2)):
        super(resSh, self).__init__()
        self.co1 = layers.Conv2D(filters, kernel_size=kerSize, padding='same', strides=stride,
                                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4))
        self.bn = layers.BatchNormalization()
        self.re = layers.Activation(activations.relu)
        self.co2 = layers.Conv2D(filters, kernel_size=kerSize, padding='same',
                                kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1e-4))
        self.co = layers.Conv2D(filters, (1, 1), strides=stride, padding='same')
    def call(self, inp):
        x = inp
        sh = inp
        x = self.co1(x)
        x = self.bn(x)
        x = self.re(x)
        x = self.co2(x)
        x = self.bn(x)
        sh = self.co(sh)
        return self.re(x+sh)

This is a resNet model based on [this paper.](https://arxiv.org/abs/1512.03385)
 
The kernel initializers and regularizers have been used with https://keras.io/zh/examples/cifar10_resnet/ as the source.

In [31]:
class resNet(tf.keras.Model):
    def __init__(self):
        super(resNet, self).__init__()
        self.co1 = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')
        self.mp1 = layers.MaxPooling2D((2, 2), padding='same')
 
        self.ri1 = resId(filters=64, kerSize=3)
        self.rs1 = resSh(filters=128, kerSize=3, stride=(2, 2))
        self.ri2 = resId(filters=128, kerSize=3)
        self.rs2 = resSh(filters=256, kerSize=3, stride=(2, 2))
        self.ri3 = resId(filters=256, kerSize=3)
        self.rs3 = resSh(filters=512, kerSize=3, stride=(2, 2))
        self.ri4 = resId(filters=512, kerSize=3)
 
        self.gap = layers.GlobalAveragePooling2D()
        self.fla = layers.Flatten()
        self.end = layers.Dense(10, activation="softmax", kernel_initializer='he_normal')
 
    def call(self, inp):
        x = inp
        x = self.co1(x)
        x = self.mp1(x)
        for i in range(3):
            x = self.ri1(x)
        x = self.rs1(x)
  #      x = layers.Dropout(0.2)(x)
        for i in range(3):
            x = self.ri2(x)
        x = self.rs2(x)
  #      x = layers.Dropout(0.2)(x)
        for i in range(5):
            x = self.ri3(x)
        x = self.rs3(x)
 #       x = layers.Dropout(0.2)(x)
        x = self.ri4(x)
        x = self.ri4(x)
   #     x = layers.Dropout(0.2)(x)
 
        x = self.gap(x)
        x = self.fla(x)
        x = self.end(x)
        return(x)
 
    def model(self):
        x = layers.Input(shape=(32, 32, 3))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

 This section is for data augmentation. Again, used from [this implementation.](https://keras.io/zh/examples/cifar10_resnet/)

In [32]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        zca_epsilon=1e-06,
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',
        horizontal_flip=True)
datagen.fit(trainIm)

In [33]:
model = resNet()
print(model.model().summary())
model.compile(  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) ,
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 16, 16, 64)   9472        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 8, 8, 64)     0           conv2d_42[0][0]                  
__________________________________________________________________________________________________
res_id_12 (resId)               (None, 8, 8, 64)     37184       max_pooling2d_3[0][0]            
                                                                 res_id_12[0][0]            

In [34]:
his = model.fit(datagen.flow(trainIm, trainLab, batch_size=32), epochs=50, callbacks=[cb], workers=4, validation_data= (testIm, testLab))

Epoch 1/50
1563/1563 [==============================] - 93s 40ms/step - loss: 2.1240 - accuracy: 0.3585 - val_loss: 2.5689 - val_accuracy: 0.1638
Epoch 2/50
1563/1563 [==============================] - 62s 39ms/step - loss: 1.7547 - accuracy: 0.4575 - val_loss: 2.4965 - val_accuracy: 0.2080
Epoch 3/50
1563/1563 [==============================] - 62s 39ms/step - loss: 1.5781 - accuracy: 0.5060 - val_loss: 2.0296 - val_accuracy: 0.3193
Epoch 4/50
1563/1563 [==============================] - 61s 39ms/step - loss: 1.5168 - accuracy: 0.5264 - val_loss: 1.9348 - val_accuracy: 0.3837
Epoch 5/50
1563/1563 [==============================] - 61s 39ms/step - loss: 1.3953 - accuracy: 0.5744 - val_loss: 1.7078 - val_accuracy: 0.4427
Epoch 6/50
1563/1563 [==============================] - 61s 39ms/step - loss: 1.3245 - accuracy: 0.6053 - val_loss: 1.6310 - val_accuracy: 0.4691
Epoch 7/50
1563/1563 [==============================] - 62s 39ms/step - loss: 1.2314 - accuracy: 0.6341 - val_loss: 1.4976 -

In [35]:
model.evaluate(testIm, testLab)

313/313 [==============================] - 4s 12ms/step - loss: 1.2182 - accuracy: 0.6637


[1.2181767225265503, 0.6636999845504761]

In [37]:
test_loss, test_acc = model.evaluate(testIm, testLab)
print("Training loss:", his.history['loss'])
print("Training accuracy:", 100*his.history['accuracy'])
print("Test loss:", test_loss)
print("Test accuracy:", 100*test_acc)

313/313 [==============================] - 4s 12ms/step - loss: 1.2182 - accuracy: 0.6637
Training loss: [2.12396240234375, 1.7546652555465698, 1.5781291723251343, 1.5167518854141235, 1.3952957391738892, 1.3245389461517334, 1.2313907146453857, 1.2042748928070068, 1.1735018491744995, 1.123280644416809, 1.1204395294189453, 1.07777738571167, 1.0283056497573853, 1.015475869178772, 0.9837028384208679, 0.9552586674690247, 0.9477741122245789, 0.9349417686462402, 0.9140617251396179]
Training accuracy: [0.35853999853134155, 0.45754000544548035, 0.5060200095176697, 0.5263599753379822, 0.5743799805641174, 0.6052799820899963, 0.6340600252151489, 0.6494399905204773, 0.6665199995040894, 0.6811000108718872, 0.6884999871253967, 0.7013400197029114, 0.7149800062179565, 0.7209200263023376, 0.7320799827575684, 0.7398200035095215, 0.7440999746322632, 0.7488800287246704, 0.7543200254440308, 0.35853999853134155, 0.45754000544548035, 0.5060200095176697, 0.5263599753379822, 0.5743799805641174, 0.60527998208999